#### 문제 5-1 : 카페 메뉴 도구(Tool) 호출 체인 구현
- 이 문제는 LangChain의 Tool Calling 기능을 학습하기 위한 기초 단계입니다. 카페 메뉴 정보를 제공하는 AI 어시스턴트를 구현하면서 다양한 데이터 소스(로컬 DB, 웹, 위키피디아)에서 정보를 검색하는 방법을 익힙니다.

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import re
import os, json

from textwrap import dedent
from pprint import pprint

import warnings
warnings.filterwarnings("ignore")

In [3]:
# 카페 메뉴 데이터 파일 생성 및 벡터 db 구축
from langchain.document_loaders import TextLoader
from langchain_core.documents import Document
import re


loader = TextLoader("../data/cafe_menu_data.txt", encoding="utf-8")
documents = loader.load()

print(len(documents))


# 문서 분할 (Chunking)
def split_menu_items(document):
    """
    메뉴 항목을 분리하는 함수 
    """
    # 정규표현식 정의 
    pattern = r'(\d+\.\s.*?)(?=\n\n\d+\.|$)'
    menu_items = re.findall(pattern, document.page_content, re.DOTALL)
    
    # 각 메뉴 항목을 Document 객체로 변환
    menu_documents = []
    for i, item in enumerate(menu_items, 1):
        # 메뉴 이름 추출
        menu_name = item.split('\n')[0].split('.', 1)[1].strip()
        
        # 새로운 Document 객체 생성
        menu_doc = Document(
            page_content=item.strip(),
            metadata={
                "source": document.metadata['source'],
                "menu_number": i,
                "menu_name": menu_name
            }
        )
        menu_documents.append(menu_doc)
    
    return menu_documents

# 메뉴 항목 분리 실행
menu_documents = []
for doc in documents:
    menu_documents += split_menu_items(doc)

# 결과 출력
print(f"총 {len(menu_documents)}개의 메뉴 항목이 처리되었습니다.")

1
총 10개의 메뉴 항목이 처리되었습니다.


In [4]:
from langchain_community.vectorstores import FAISS
from langchain.embeddings import OllamaEmbeddings

embeddings_model = OllamaEmbeddings(model="bge-m3:latest") 

# FAISS 인덱스 생성
cafe_db = FAISS.from_documents(
    documents=menu_documents, 
    embedding=embeddings_model
)

# FAISS 인덱스 저장 (선택사항)
cafe_db.save_local("../db/cafe_db")


# Retriever 생성
menu_retriever = cafe_db.as_retriever(
    search_kwargs={'k': 2},
)

C:\Users\yondu\AppData\Local\Temp\ipykernel_15396\2750516900.py:4: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings_model = OllamaEmbeddings(model="bge-m3:latest")


In [5]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [8]:
from langchain.tools import tool
import wikipedia
from langchain_community.vectorstores import FAISS
from langchain.tools import tool
from langchain_community.tools import TavilySearchResults
from langchain_core.tools import tool


@tool
def tavily_search_func(query: str) -> str:
    """
    Search the internet for the latest or external information using Tavily.
    """
    tavily_search = TavilySearchResults(max_results=2)
    docs = tavily_search.invoke(query)

    if not docs:
        return "관련 정보를 찾을 수 없습니다."

    formatted_docs = "\n---\n".join([
        f'<Document href="{doc["url"]}"/>\n{doc["content"]}\n</Document>'
        for doc in docs
    ])

    return formatted_docs


# b. Wikipedia 요약 도구
@tool
def wiki_summary(topic: str) -> str:
    """Wikipedia에서 일반 정보를 검색해 요약합니다"""
    try:
        return wikipedia.summary(topic, sentences=3)
    except wikipedia.exceptions.DisambiguationError as e:
        return f"'{topic}'은(는) 모호한 주제입니다. 가능한 항목: {', '.join(e.options[:5])}"
    except wikipedia.exceptions.PageError:
        return "위키피디아에 해당 주제가 없습니다."


# c. FAISS 벡터 DB 검색 도구
@tool
def db_search_cafe_func(query: str) -> str:
    """
    카페 메뉴 벡터 DB에서 유사한 항목을 검색하여 텍스트로 반환합니다.
    """
    docs = cafe_db.similarity_search(query, k=2)
    if docs:
        return "\n\n".join([doc.page_content for doc in docs])
    return "관련 메뉴 정보를 찾을 수 없습니다."


# LLM에 도구를 바인딩 (2개의 도구 바인딩)
llm_with_tools = llm.bind_tools(tools=[tavily_search_func, wiki_summary, db_search_cafe_func])


In [13]:
from langchain_core.runnables import Runnable, RunnableLambda, RunnableConfig
from langchain_core.runnables import chain
from langchain_core.messages import HumanMessage

from langchain_core.runnables import chain

@chain
def tool_chain(inputs: dict, config: RunnableConfig = None) -> str:
    """
    사용자 질문을 처리하고 적절한 도구를 호출한 후 최종 답변을 생성하는 체인
    """
    question = inputs["question"]

    # LLM이 어떤 도구를 사용할지 결정
    response = llm_with_tools.invoke(question)

    # 도구가 호출된 경우: tool_calls 속성 확인
    if hasattr(response, "tool_calls") and response.tool_calls:
        tool_outputs = []
        for call in response.tool_calls:
            tool_name = call["name"]
            tool_args = call["args"]
            
            if tool_name == "db_search_cafe_func":
                result = db_search_cafe_func.invoke(tool_args["query"])
                tool_outputs.append(result)
            elif tool_name == "tavily_search_func":
                result = tavily_search_func.invoke(tool_args["query"])
                tool_outputs.append(result)
            elif tool_name == "wiki_summary":
                result = wiki_summary.invoke(tool_args["topic"])
                tool_outputs.append(result)

        # 도구 결과를 LLM에 다시 전달하여 최종 응답 생성
        followup_prompt = f"질문: {question}\n도구 결과:\n" + "\n".join(tool_outputs)
        final_answer = llm.invoke(followup_prompt)
        return final_answer.content
    
    # 도구 호출 없이도 답할 수 있는 경우
    return response.content


# 질문 입력
test_input = {"question": "아메리카노의 가격과 특징은 무엇인가요?"}
answer = tool_chain.invoke(test_input)

print("답변:\n", answer)

답변:
 아메리카노의 가격과 특징은 다음과 같습니다:

### 아메리카노
- **가격:** ₩4,500
- **주요 원료:** 에스프레소, 뜨거운 물
- **설명:** 진한 에스프레소에 뜨거운 물을 더해 만든 클래식한 블랙 커피입니다. 원두 본연의 맛을 가장 잘 느낄 수 있으며, 깔끔하고 깊은 풍미가 특징입니다. 설탕이나 시럽 추가가 가능합니다.

### 아이스 아메리카노
- **가격:** ₩4,500
- **주요 원료:** 에스프레소, 차가운 물, 얼음
- **설명:** 진한 에스프레소에 차가운 물과 얼음을 넣어 만든 시원한 아이스 커피입니다. 깔끔하고 시원한 맛이 특징이며, 원두 본연의 풍미를 느낄 수 있습니다. 더운 날씨에 인기가 높습니다.

아메리카노는 기본적으로 에스프레소를 기반으로 하여, 물의 비율에 따라 농도를 조절할 수 있는 커피 음료입니다.


#### 문제 5-2 : Few-shot 프롬프팅을 활용한 카페 AI 어시스턴트 
- 문제 1의 기본 체인을 발전시켜, Few-shot 프롬프팅 기법을 적용한 고급 AI 어시스턴트를 구현합니다. 이를 통해 AI가 언제 어떤 도구를 사용해야 하는지 더 정확하게 판단할 수 있도록 합니다.

In [15]:
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent

# Few_shot 프롬프트 활용
examples = [
    HumanMessage(content="아메리카노 정보와 커피 역사를 알려주세요.", name="example_user"),
    AIMessage(content="카페 메뉴 정보 검색과 위키피디아 검색을 진행하겠습니다.", name="example_assistant"),
    AIMessage(content="", name="example_assistant", tool_calls=[
        {"name": "db_search_cafe_func", "args": {"query": "아메리카노"}, "id": "1"}
    ]),
    ToolMessage(content="아메리카노: 에스프레소에 뜨거운 물을 추가한 커피. 부드럽고 깔끔한 맛이 특징.", tool_call_id="1"),
    AIMessage(content="", name="example_assistant", tool_calls=[
        {"name": "wiki_summary", "args": {"query": "커피의 역사", "k": 1}, "id": "2"}
    ]),
    ToolMessage(content="커피는 에티오피아에서 기원해 아랍 세계를 거쳐 유럽과 전 세계로 퍼졌습니다. 15세기 수피 교단에서 각성 효과로 즐겼습니다.", tool_call_id="2"),
    AIMessage(content=(
        "아메리카노는 에스프레소에 뜨거운 물을 추가하여 만든 커피로, 부드럽고 깔끔한 맛이 특징입니다. "
        "커피는 에티오피아에서 유래하여 아랍 세계를 거쳐 유럽과 세계로 퍼졌으며, 초기에는 수피 교단에서 각성제로 활용되기도 했습니다."
    ), name="example_assistant")
]
system_message = """You are an AI assistant that answers questions about café menu items and coffee-related knowledge.
- Use the `db_search_cafe_func` tool for menu information.
- Use the `wiki_summary` tool for general coffee knowledge.
- You may chain tools logically to gather necessary information before replying."""

few_shot_prompt = ChatPromptTemplate.from_messages([
    ("system", system_message),
    *examples,
    ("human", "{query}"),
])

llm = ChatOpenAI(model="gpt-4o-mini")
tools = [wiki_summary, db_search_cafe_func, tavily_search_func]
#llm_with_tools = llm.bind_tools(tools=tools)
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

#fewshot_search_chain = few_shot_prompt | llm_with_tools
query = "콜드브루 정보와 유래를 알려줘"
#response = fewshot_search_chain.invoke({"query": query})

response = agent.run(query)
print(response)

#for tool_call in response.tool_calls:
#    print(tool_call)

C:\Users\yondu\AppData\Local\Temp\ipykernel_15396\2614486150.py:37: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)
C:\Users\yondu\AppData\Local\Temp\ipykernel_15396\2614486150.py:43: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` 



> Entering new AgentExecutor chain...
콜드브루에 대한 일반적인 정보와 유래를 알아보려면, 위키피디아에서 콜드브루에 대한 내용을 검색해야 할 것 같습니다.  
Action: wiki_summary  
Action Input: "콜드브루"  
Observation: 위키피디아에 해당 주제가 없습니다.
Thought:콜드브루에 대한 정보를 다른 방법으로 찾아야 할 것 같습니다. 카페 메뉴에서 관련된 정보를 검색해보겠습니다.  
Action: db_search_cafe_func  
Action Input: "콜드브루"  
Observation: 6. 콜드브루
   • 가격: ₩5,000
   • 주요 원료: 콜드브루 원액, 차가운 물
   • 설명: 찬물에 12-24시간 우려낸 콜드브루 원액을 사용한 시원한 커피입니다. 부드럽고 달콤한 맛이 특징이며, 산미가 적어 누구나 부담 없이 즐길 수 있습니다. 얼음과 함께 시원하게 제공됩니다.

1. 아메리카노
   • 가격: ₩4,500
   • 주요 원료: 에스프레소, 뜨거운 물
   • 설명: 진한 에스프레소에 뜨거운 물을 더해 만든 클래식한 블랙 커피입니다. 원두 본연의 맛을 가장 잘 느낄 수 있으며, 깔끔하고 깊은 풍미가 특징입니다. 설탕이나 시럽 추가 가능합니다.
Thought:콜드브루에 대한 기본적인 정보는 얻었지만, 유래에 대한 내용이 부족합니다. 콜드브루의 유래를 알아보기 위해 추가적인 정보를 검색해야 할 것 같습니다.  
Action: tavily_search_func  
Action Input: "콜드브루 유래"  

C:\Users\yondu\AppData\Local\Temp\ipykernel_15396\47104782.py:14: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  tavily_search = TavilySearchResults(max_results=2)



Observation: <Document href="https://www.nespresso.com/pro/kr/ko/story/coldbrew"/>
콜드브루의 유래는 정확하게 기록된 바가 없지만, 가장 유력한 두 가지 설이 있어요. 하나는 네덜란드가 인도네시아를 식민 지배할 때 네덜란드 선원들이 장기간 항해 중
</Document>
---
<Document href="https://m.blog.naver.com/cafe_awesomebrew/222511600073"/>
▪️▫️▪️

​

**2️⃣ 더치커피 콜드브루의 유래인 네덜란드에는 더치커피가 없다? 는데..**

![](https://mblogthumb-phinf.pstatic.net/MjAyMTA5MjBfMTQw/MDAxNjMyMTIyNTkyNzI1.r-iArf0Y00neYeKi5Jks5fkJvbvVSG_LC4b_78PJHXsg._lef56Gp_90auy9ozsdbs2xhaH7-9KbcOMEX2-zNfEYg.PNG.cafe_awesomebrew/20210920%EF%BC%BF162218%EF%BC%BF0002.png?type=w80_blur)

​

영어권에서는 더치커피라는 용어가 존재한 적이 아예 없다.

위키백과에서 찾아봐도 "dutch coffee" 란 항목이 없다.

구글 검색해도 찬물로 우려내는 dutch coffee는 없다.

​

스타벅스나 스텀프타운에서도 같은 방식인 콜드브루 커피를 론칭하는 등

실제로 유통되는 커피의 한 종류인데 [...] **Ⅰ. 더치커피의 유래ㆍ콜드브루는 어떻게 탄생하게 되었을까?**

**​**

**Ⅱ. 더치커피 콜드브루의 유래인 네덜란드에는 더치커피가 없다? 는데..**

**​**

**Ⅲ. 콜드브루와 더치커피는 어떤 차이가 있나? 같은 의미일까?**

![](https://mblogthumb-phinf.pstatic.net/MjAyMTA5MjBfMTU2/MDAxNjMyMDY4MzQ1Njc1.BA_D7zd9rBqSZZ-kDSjCRHm1diR4xMzE_DPr